<a href="https://colab.research.google.com/github/hwarang97/Pytorch_introduction/blob/main/model_fitting_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 매개변수 최적화하기
- 모델에 데이터를 학습, 검증, 테스트하는 단계
- 설정된 epoch만큼 진행
- 예측과 정답사잉의 오차(손실함수)로 모델 성능 측정
- 최적화 진행

## 지금까지 코드 정리

In [1]:
from torch.nn.modules.activation import Softmax
import torch
from torch import nn # 계층의 모든 구성 요소 제공
from torch.utils.data import DataLoader # 데이터를 이터러블한 객체로 만들어줌
from torchvision import datasets # 사용할 데이터 제공(PIL)
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,         # root에 데이터가 없다면, 인터넷으로 다운로드 진행
    transform=ToTensor()   # 이미지(PIL Image), 넘파이 배열을 floatTensor로 변환, 픽셀값을 0~1사이로 정규화시켜줌
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,         # root에 데이터가 없다면, 인터넷으로 다운로드 진행
    transform=ToTensor()   # 이미지(PIL Image), 넘파이 배열을 floatTensor로 변환, 픽셀값을 0~1사이로 정규화시켜줌
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512,10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork()

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



## 하이퍼파라미터
- 모델 설정 값(사용자가 직접 지정)
- 어떤 값을 설저해주냐에 따라 성능이 달라짐
- 대표적인 하이퍼파라미터
  - epoch : 데이터셋 반복수
  - batch size : 한번 학습에 사용하는 데이터 양
  - learning rate : 매개변수 조절하는 정도

In [2]:
learning_rate = 1e-3 # 0.001
batch_size = 64
epochs = 5

## 최적화 단계(Optimization Loop)
- 하이퍼 파라미터를 설정한 뒤, 최적화 단계 진행
- 최적화 단계 반복수를 epoch로 설정
- 최적화 단계
  - 학습 단계(train loop) - 학습용 데이터셋을 반복하고 최적의 매개변수로 수렴
  - 검증/테스트 단계(validation/test loop) - 모델 성능이 개선되고 있는지를 확인하기 위한 단계

## 손실 함수(loss function)
- 모델의 예측과 정답 사이의 틀린 정도를 제공
- 학습을 통해 손실 함수 값을 최소화 시키는 것이 모델 학습의 목표
- 회귀 문제에 사용되는 손실 함수
  - MSE
  - MAE
- 분류 문제에 사용되는 손실 하숨
  - NLLLoss
  - CrossEntropyLoss

In [3]:
# 손실 함수 초기화
loss_fn = nn.CrossEntropyLoss()

## 옵티마이저(Optimizer)
- 최적화 시켜주는 객체
- 종류
  - SGD
  - ADAM
  - RMSProp

In [4]:
# 학습하려는 모델의 매개변수와 학습률, 하이퍼라라미터를 등록하여 옵티파이저 초기화
oprimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## 학습 관련 코드 구현

In [14]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader, 1): # 64개의 이미지 세트를 구조, 이미지 하나씩 넣어서 돌리고 
                                              # batch = 100번째(이미지로는 64 * 100번째) 마다 손실함수랑 성능값을 알려줘
    """예측과 손실 계산"""
    pred = model(X) # 이미지 64개를 한번에 넣기
    loss = loss_fn(pred, y)

    """최적화"""
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X) # batch * 64
      print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]') 

def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10   
for t in range(epochs):
  print(f'Epoch {t+1}\n-----------------------------------')
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print('Done!!!')

Epoch 1
-----------------------------------
loss: 0.545640 [    0/60000]
loss: 0.646358 [ 6400/60000]
loss: 0.433265 [12800/60000]
loss: 0.689584 [19200/60000]
loss: 0.610146 [25600/60000]
loss: 0.584947 [32000/60000]
loss: 0.618011 [38400/60000]
loss: 0.682079 [44800/60000]
loss: 0.659936 [51200/60000]
loss: 0.597318 [57600/60000]
Test Error: 
 Accuracy: 79.1%, Avg loss: 0.598955 

Epoch 2
-----------------------------------
loss: 0.533814 [    0/60000]
loss: 0.634516 [ 6400/60000]
loss: 0.424029 [12800/60000]
loss: 0.680900 [19200/60000]
loss: 0.603424 [25600/60000]
loss: 0.578879 [32000/60000]
loss: 0.607457 [38400/60000]
loss: 0.678098 [44800/60000]
loss: 0.655100 [51200/60000]
loss: 0.587872 [57600/60000]
Test Error: 
 Accuracy: 79.4%, Avg loss: 0.590659 

Epoch 3
-----------------------------------
loss: 0.522840 [    0/60000]
loss: 0.623637 [ 6400/60000]
loss: 0.415588 [12800/60000]
loss: 0.672677 [19200/60000]
loss: 0.596937 [25600/60000]
loss: 0.573260 [32000/60000]
loss: 0.59